In [7]:
# 책 내용 대신 파이썬 기본 smtplib, email, imaplib 모듈 활용
import smtplib
import email
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import imaplib
from pathlib import Path
import pyinputplus as pyip

In [2]:
# smtplib.SMTP()함수를 통해 smtp 객체 생성(587 포트 활용)
# 587 포트가 안될 경우 smtplib.SMTP_SSL('smtp.gmail.com', 465) 활용
smtpObj = smtplib.SMTP('smtp.gmail.com', 587)
# smtp 객체를 생성한 후에는 ehlo() 메서드를 제일 먼저 호출해야(헬로 도메인)
smtpObj.ehlo()
# 연결을 암호화 하기 위하여 starttls() 메서드 호출 -> SSL을 사용하여 465 포트에 연결했다면 이 단계 건너뛰기 가능
smtpObj.starttls()

(220, b'2.0.0 Ready to start TLS')

In [3]:
# pyinputplus를 활용하여 소스코드에 비밀번호 직접 넣지 않도록 조치
password = pyip.inputStr(prompt='비밀번호를 입력해주세요.: ')
# login 메서드를 이용해 로그인 -> 반환값 235는 인증에 성공했음을 의미
smtpObj.login('kth1149@gmail.com', password)
password = None

비밀번호를 입력해주세요.: 

In [4]:
# MIMEMultipart() 및 MIMEText() 이용하여 MIMEText(메시지) 등 객체 생성
# MIMEText()에 html 문서를 집어넣을 수도 있으며, 그 경우 두 번째 인자로 'html' 넣어줘야 -> HTML 웹에디터 활용 가능
msg = MIMEMultipart()
# msg['subject'], msg['to'], msg['from'], msg['cc'], msg['bcc'] 활용
msg['Subject'] = '파이썬에서 보내는 이메일입니다.'
msg['To'] = 'tkim@iportfolio.co.kr'
msg['From'] = 'kth1149@gmail.com'
msg['cc'] = 'kth1149@gmail.com'
body = '파이썬에서 보내는 이메일입니다.'
# attach 메서드를 활용하여 Multipart()에 MIMEText() 객체 삽입
msg.attach(MIMEText(body))

In [5]:
# sendmail() 메서드를 통해 메일 보내기(보내는 사람, 받는 사람, 메시지 내용 순서의 인자)
# msg.as_string()을 활용하여 설정되었던 'cc' 주소로도 메일을 보낼 수 있음을 확인
smtpObj.sendmail(msg['from'], msg['to'], msg.as_string())

{}

In [6]:
# quit() 메서드를 사용해 연결 종료
smtpObj.quit()

(221,
 b'2.0.0 closing connection a4-20020a170902ecc400b0016d2e772550sm6995609plh.175 - gsmtp')